In [1]:
!pip install qwen-vl-utils[decord]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.5/39.5 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 40.8 MB/s eta 0:00:00


In [2]:
from IPython.display import Image

In [3]:
import torch
from transformers import Qwen2VLForConditionalGeneration,AutoProcessor
from qwen_vl_utils import process_vision_info

In [ ]:
model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-7B-Instruct", torch_dtype="auto", device_map="auto"
)
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-7B-Instruct")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/56.5k [00:00<?, ?B/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/3.90G [00:00<?, ?B/s]

In [ ]:
def find_stage_column_concreting(image):
  messages = [
        {
            "role": "user",
            "content": [
                {"type": "image", "image": image},
                {"type": "text", "text": "Analyze the image and detect the stage of construction of Column Concreting with in three phases namely (1.yet to start,2.work in progress,3.completed)"},
                {"type": "text", "text": "1. For yet to start phase conditions are (No Reinforcement Visible /No Shuttering or Formwork Visible/Foundation/Stump Only/No Construction Materials for Columns Present)"},
                {"type": "text", "text": "2. For Work in Progress phase conditions are (Reinforcement in Place, Shuttering/Formwork Erected, Concrete Pouring Apparatus, Fresh or Partially Cured Concrete, Partial Removal of Formwork)."},
                {"type": "text", "text": "3. For Completed phase conditions are (Formwork Fully Removed,Next Stage Shuttering,Cured Column Concrete,Additional Structural Elements Above)."},
            ],
        }
    ]
  text = processor.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
  image_inputs, video_inputs = process_vision_info(messages)
  inputs = processor(
        text=[text],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt",
    )
  inputs = inputs.to("cuda")
  generated_ids = model.generate(**inputs, max_new_tokens=128)
  generated_ids_trimmed = [out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)]
  output_text = processor.batch_decode(generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False)
  return output_text[0]

In [ ]:
image1 = '/content/131.jpg'
image2 = '/content/132.jpg'
image3 = '/content/133.jpg'
image4 = '/content/134.jpg'

In [ ]:
Image(filename=image1)

In [ ]:
test1 = find_stage_column_concreting(image1)
test1

In [ ]:
Image(filename=image2)

In [ ]:
test2 = find_stage_column_concreting(image2)
test2

In [ ]:
Image(filename=image3)

In [ ]:
test3 = find_stage_column_concreting(image3)
test3

In [ ]:
Image(filename=image4)

In [ ]:
test4 = find_stage_column_concreting(image4)
test4